# Objective
Build a rhythm database.<br>
Compare the target's rhythm with the rhythm database.<br>
Create chords for the target based on the key and the matched rhythm.<br>

# 1. Build Rhythm Database
*Q: Build for what?<br>Ans: Build to compare the rhythms, find the best-matched pattern that fits the target.*

Current implementation:<br>
**Piano with 'left-hand' channel**<br>
**A note cannot across multiple bars**

Flow, for-each *.mid:<br>
a. Get the 'left-hand' channel, which usually is the accompany.<br>
b. Append each bar from 'left-hand' into the note-list.<br>
c. Remove duplicate.<br>
d. Save with pickle.<br>
e. (Optional) Optimise query speed with locality sensitive hashing. <br>

# 2. Extract target's rhythm *from the accompany channel(s)*
*p.s. Drum is not considered an accompany*

Definition:<br>
Rhythm = note distribution in **a single** bar.

Flow:<br>
a-i. Find the accompany channel(/s), which is(/are) lower than pitch=54.

In [48]:
from miditoolkit.midi import parser as mid_parser

def find_accompany(piece_name, threshold_pitch=54):
    '''
    Find the accompany channels which are lower than the threshold
    
    Parameters
    ----------
    piece_name : str
        The name of the midi file.
    threshold_pitch: int
        The maximum pitch for accompany channels.

    Returns
    -------
    list
        The list of index of the chosen channels
    '''
    
    # Parse midi file
    mido_obj = mid_parser.MidiFile(piece_name)
    
    chosen_channel = []
    # Choose the "appropriate" channel(s)
    for idx, inst in enumerate(mido_obj.instruments):
        # Drum is not considered an accompany
        if inst.is_drum: continue
            
        # Calculate the average pitch
        total_pitch = 0
        total_note = 0
        for note in inst.notes:
            total_pitch += note.pitch
            total_note += 1
        avg_pitch = total_pitch/total_note
        # Choose only if the pitch is lower than threshold
        if avg_pitch <= threshold_pitch:
            chosen_channel.append(idx)
    
    # At least 1 channel should be chosen. 
    # If not, choose the first channel.
    if chosen_channel == []:
        chosen_channel = [0]
        
    return chosen_channel

a-ii. Combine all notes from the chosen channels into 1-d list.

In [50]:
def combine_accompany(piece_name, accompany_idx):
    '''
    Combine the chosen accompany channel(s).
    
    Parameters
    ----------
    piece_name : str
        The name of the midi file.
    idx: list (of int)
        The list of index of the chosen channels

    Returns
    -------
    list
        The 1-d list of combined channels
    '''
    # Parse midi file
    mido_obj = mid_parser.MidiFile(piece_name)
    
    final_notelist = []
    # Combine the channels
    for channel in accompany_idx:
        for note in mido_obj.instruments[channel].notes:
            final_notelist.append(note)
    final_notelist = sorted(final_notelist,key = lambda x:x.start)
    return final_notelist

b. Find the rhythm. 

*Q: What does class lhMatchInstance stand for?*<br>
*A: ?*

Just regard this as a black box:<br>
b-i. Get time signature.<br>
b-ii. Get each bar.<br>
b-iii. Transform into rhythm.<br>
| input=notes | -> | output=rhythm |<br>

# 3. Extract target's chord information
-> chord-identification/.../pipeline.ipynb

# 4. Finalise
Flow:<br>
a. Extract the melody.<br>
b. Match the rhythm.<br>
c. Add chords.<br>
d. Combine and output.<br>

# Post-feedback
Notice that you use LSTM/HMM to estimate the key, I would like to provide one more option.

***Krumhansl-Schmuckler key-finding algorithm***

It is a well-known statistic method in cognitive science, calculating the correlation of the key distribution. It should be the most lightweight method (if being lightweight is a key factor in the project).

A well-described document: http://rnhart.net/articles/key-finding/